# [Regression trials](#regression-trials)

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from functools import reduce
from io import StringIO

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from azure.storage.blob import BlobServiceClient
from collections import namedtuple
from IPython.display import display, display_html
from sklearn.compose import ColumnTransformer
from sklearn import linear_model
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.model_selection import (
    cross_validate,
    GridSearchCV,
    KFold,
    PredefinedSplit,
    RepeatedKFold,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    PolynomialFeatures,
    PowerTransformer,
    QuantileTransformer,
    StandardScaler,
)
from yellowbrick.model_selection import LearningCurve

In [ ]:
%aimport src.visualization_helpers
from src.visualization_helpers import (
    plot_corr_map,
    plot_coef_plot,
    plot_cv_scores,
    plot_nested_cv,
    plot_multiple_histograms,
    plot_pairplot,
    plot_qq,
    plot_residual_manually,
    show_yb_learning_curves,
    show_yb_prediction_error,
    show_yb_residual_plot,
    plot_multi_feature_target,
    plot_single_feature_target,
    plot_multiple_bar_charts,
    plot_side_by_side_bar_box,
)

%aimport src.metrics_helpers
from src.metrics_helpers import get_test_metrics

%aimport src.ml_helpers
from src.ml_helpers import (
    generate_dummies,
    filter_by_value_counts,
    get_model_coefs,
    DFDropDupicates,
    DFFilterNumerical,
    DFPolynomialFeatureGenerator,
    DFInteractionTerms,
    DFDropDupicates,
    DFDropNa,
    DFColTypeChanger,
    DFOneHotEncoder,
    DFStandardScaler,
)

In [ ]:
SMALL_SIZE = 20
MEDIUM_SIZE = 22
BIGGER_SIZE = 24
plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["axes.facecolor"] = "white"
sns.set_style("darkgrid", {"legend.frameon": False})
sns.set_context("talk", font_scale=0.95, rc={"lines.linewidth": 2.5})

In [ ]:
pd.set_option("display.max_rows", 750)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

<a id="toc"></a>

## [Table of Contents](#table-of-contents)
0. [About](#about)
1. [User Inputs](#user-inputs)
2. [Load Price listing data and split](#load-price-listing-data-and-split)
3. [Exploratory Data Analysis on target variable](#exploratory-data-analysis-on-target-variable)
4. [Applying filters to drop outliers](#applying-filters-to-drop-outliers)
5. [Pre-processing](#pre-processing)
6. [Preparing for modeling and feature selection](#preparing-for-modeling-and-feature-selection)
7. [Base model - numerical features only](#base-model-numerical-features-only)
   - 7.1. [Raw features](#raw-features)
   - 7.2. [Transformed features](#transformed-features)
     - 7.2.1. [Quantile transformation](#quantile-transformation)
     - 7.2.2. [Yeo-Johnson transformation](#yeo-johnson-transformation)
     - 7.2.3. [`log1plus` transformation](#log1plus-transformation)
   - 7.3. [Polynomial and Interaction terms](#polynomial-and-interaction-terms)
8. [Include categorical features](#include-categorical-features)
   - 8.1. [Best categorical features](#best-categorical-features)
   - 8.2. [Summary of adding categorical features](#summary-of-adding-categorical-features)
9. [Assessing best model](#assessing-best-model)
   - 9.1. [Extracting best model](#extracting-best-model)
   - 9.2. [Get best model coefficients](#get-best-model-coefficients)
   - 9.3. [Residuals for predictions](#residuals-for-predictions)
     - 9.3.1. [Residual distributions](#residual-distributions)
     - 9.3.2. [Worst performing predictions](#worst-performing-predictions)
     - 9.3.3. [Residuals separated by State](#residuals-separated-by-state)
     - 9.3.4. [Quantile-quantile plot](#quantile-quantile-plot)
   - 9.4. [Permutation Importances](#permutation-importances)
   - 9.5. [Prediction error](#prediction-error)
   - 9.6. [Summary of scoring metrics](#summary-of-scoring-metrics)
     - 9.6.1. [Comparison of ordinary linear regression between `sklearn` and `statsmodels`](#comparison-of-ordinary-linear-regression-between-sklearn-and-statsmodels)
       - 9.6.1.1. [Using `statsmodels`](#using-statsmodels)
       - 9.6.1.2. [Using `sklearn`](#using-sklearn)
   - 9.7. [Check influence of choice of test split](#check-influence-of-choice-of-test-split)
   - 9.8. [Check of bias and variance](#check-of-bias-and-variance)
10. [Future work](#future-work)

<a id="about"></a>

## 0. [About](#about)

In this notebook, we will experiment with regression models on the processed car listings data in `data/processed_*.csv`. As mentioned in the `README.md` file, the objective is to build a linear regression model that can use listed features about cars listed to predict the listing price to within \$5,000.

**Notes**
1. Throughout this analysis, we will refer to the requirements of linear regression, which can be found in this [Duke guide](http://people.duke.edu/~rnau/testing.htm).
2. This analysis will be limited to the following (`v1`, version 1) set of columns from the data (at the most)
   - `Mileage`
   - `MPG`
   - `consumer_reviews`
   - `seller_reviews`
   - `Fuel Type`
   - `Drivetrain`
   - `State`
   - `seller_rating`
   - `consumer_stars`
   - `Comfort`
   - `Performance`
   - `Exterior Styling`
   - `Interior Design`
   - `Reliability`
   - `year`
   - `make`
   - `trans_speed`

<a id="user-inputs"></a>

## 1. [User Inputs](#user-inputs)

We'll define lists containing the names of features to be used in the regression modelling. Separate lists will be created for numerical and categorical features
- some are commented as engineered feature, and will be discussed below

In [ ]:
# Numerical columns
nums = [
    "Mileage",
    "MPG",  # "Highway MPG" + "City MPG",
    "consumer_reviews",
    "seller_reviews",
    # "tank_volume",  # higher RMSE when included instead of MPG; correlated to MPG, so can't combine
]

# Categorical columns
cats = [
    "Fuel Type",
    "Drivetrain",
    #     # "Exterior Color",  # very high cardinality, so exclude
    #     # "Interior Color",  # very high cardinality, so exclude
    "State",  # consider this to be a proxy for ["seller_zip" + "City"]
    #     # "seller_rating",  # RMSE on validation set blows up for LinearRegression
    #     # "consumer_stars",  # RMSE on validation set blows up for LinearRegression
    "Comfort",  # nested CV and learning curves blow up for LinearRegression
    #     # "Performance",  # slightly higher RMSE when this is included, so exclude
    "Exterior Styling",  # nested CV and learning curves blow up for LinearRegression
    "Interior Design",  # nested CV and learning curves blow up for LinearRegression
    #     # "Reliability",  # slightly higher RMSE when this is included, so exclude
    "year",
    #     # "model",  # very high cardinality, so exclude
    "trans_speed",
]

# Number of inner and outer trials for nested cross-validation
NUM_NESTED_CV_TRIALS = 5

In [ ]:
all_selected = nums + cats

In [ ]:
# The following dictionaries are not used in analysis here
# # Dictionary to define feature interactions for custom pipeline transformers
interact = {
    "mpg05Xtank_volume": {"f1": "MPG05", "f2": "tank_volume"},
    "mileageXtank_volume": {"f1": "Mileage", "f2": "tank_volume"},
    "mpg05Xconsumer_reviews": {"f1": "MPG05", "f2": "consumer_reviews"},
}
# # Dictionary to define filters for categorical columns for custom pipeline transformers
vc = {
    "vc_filter_fuel_type": {
        "col": "Fuel Type",
        "numerical_value": 10,
        "filter_type": "vc_lt",
    },
    "vc_filter_trans_speed": {
        "col": "trans_speed",
        "numerical_value": 10,
        "filter_type": "vc_lt",
    },
    "vc_filter_make": {"col": "make", "numerical_value": 10, "filter_type": "vc_lt"},
    "vc_filter_seller_rating": {
        "col": "seller_rating",
        "numerical_value": [3.8, 3.7, 4.0],
        "filter_type": "vc_isin",
    },
    "vc_filter_consumer_stars": {
        "col": "consumer_stars",
        "numerical_value": 10,
        "filter_type": "vc_lt",
    },
}

In [ ]:
def rmse_scorer(y_true, y_pred):
    return -1 * mean_squared_error(y_true, y_pred, squared=False)

<a id="load-price-listing-data-and-split"></a>

## 2. [Load Price listing data and split](#load-price-listing-data-and-split)

We'll start by loading the car listings price data into a `DataFrame`

In [ ]:
az_storage_container_name = "myconedesx7"
az_blob_name = "blobedesz17"
conn_str = (
    "DefaultEndpointsProtocol=https;"
    f"AccountName={os.getenv('AZURE_STORAGE_ACCOUNT')};"
    f"AccountKey={os.getenv('AZURE_STORAGE_KEY')};"
    f"EndpointSuffix={os.getenv('ENDPOINT_SUFFIX')}"
)
blob_service_client = BlobServiceClient.from_connection_string(conn_str=conn_str)
blob_client = blob_service_client.get_blob_client(
    container=az_storage_container_name, blob=az_blob_name
)
blobstring = blob_client.download_blob().content_as_text()
df = pd.read_csv(StringIO(blobstring))
display(df.head(2))
print(df.shape)

The data will now be split into training and testing sets

In [ ]:
X = df.drop(["price"], axis=1)
y = df["price"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
train_indices = X_train.index

There are approximately 500 more records for `TX` than for `WA`

In [ ]:
X_train["State"].value_counts().to_frame()

<a id="exploratory-data-analysis-on-target-variable"></a>

## 3. [Exploratory Data Analysis on target variable](#exploratory-data-analysis-on-target-variable)

To start exploration of the dataset, the distribution of the target variable - the listing price - is shown below

In [ ]:
ax = sns.distplot(y_train)
fig = plt.gcf()
ax.set_title("Histogram of listing prices", loc="left", fontweight="bold")

**Observations**
1. Although generally shaped similar to a normal distribution, we observe stronger (higher) tails and broader width for than for a normal distribution. Also there is a left skew that could introduce non-linearities - taking a `log` would have been beneficial for a right skew, but would only further skew an already existing left skew as is the case here.
2. A weak bimodal shape appears in the distribution. The two apparent peaks centered at approx. USD 25,000 and USD 37,000 appear to be partly merged together creating this effect. We could filter the data based on this to remove the bimodal features, but this would introduce selection bias since we don't know the test set distribution or, more importantly, the distribution of data to be used for inference (if such a model were to be considered in a tool put into production). Instead, the raw `price` target data will be comapred against transformed versions, in linear regression models, and the best transformation or the untransformed version of the data (based on lowest cross-validation error score) will be used. This could also be further investigated by scraping more data to determine if the two apparent peaks can be more clearly separated.

<a id="applying-filters-to-drop-outliers"></a>

## 4. [Applying filters to drop outliers](#applying-filters-to-drop-outliers)

We'll first de-select rows where the numerical feature `per_month_min` is zero. These correspond to listings where the monthly payment is listed as zero dollars which is unrealistic. The relationship between these two `price`-based features (`per_month_min` and `price`) is shown below

In [ ]:
non_zero_pmm_idx = X_train[X_train["per_month_min"] != 0].index
plot_corr_map(
    df=pd.concat(
        [y_train.loc[non_zero_pmm_idx], X_train.loc[non_zero_pmm_idx, "per_month_min"]],
        axis=1,
    ),
    annot=True,
    annot_fmt=".3f",
    fig_size=(2, 2),
)

**Observations**
1. It is clear that the `per_month_min` financing option is determined from the listing `price`. So, is a perfect correlation to price. So we will exclude this feature in our models in order to avoid the problem of multi-collinearity in linear regression when interpreting linear model coefficients.

We'll apply two filters to the numerical features `MPG` and `Mileage`, in the training data, in order to remove outliers

In [ ]:
X_train_filtered = X_train[(X_train["MPG"] < 2000) & (X_train["Mileage"] < 10000)]

We'll now visualize the relationship between the numerical features and the target `price` in the filtered training data

In [ ]:
df_train_filtered = pd.concat([X_train_filtered, y_train], axis=1)
nums_all = ["Mileage", "MPG", "consumer_reviews", "seller_reviews", "tank_volume"]

In [ ]:
plot_multi_feature_target(
    df_train_filtered,
    ["MPG", "price", "State", "make", "year"],
    ["price"],
    nums_all,
    label_font_size=12,
    title_font_size=12,
    marker_size=80,
    marker_opacity=0.5,
    marker_linewidth=0.5,
    color_by_col="State",
    legend_vertical_offset=-5,
    fig_size=(250, 300),
)

**Observations**
1. Only one numerical feature `MPG` has a reasonable linear relationship to the target `price`
   - it requires further filtering to remove outliers, about approx. 80 MPG

We'll look at bar charts of all the categorical features chosen for this workflow ([version 1](#about))

In [ ]:
cats_cols_bar_chart = [
    "Fuel Type",
    "Drivetrain",
    "State",
    "seller_rating",
    "consumer_stars",
    "Comfort",
    "Performance",
    "Exterior Styling",
    "Interior Design",
    "Reliability",
    "year",
    "make",
    "trans_speed",
]

In [ ]:
plot_multiple_bar_charts(
    df_train_filtered, cats_cols_bar_chart, 20, 3, -3, 12, 12, (300, 350)
)

**Observations**
1. `Fuel Type` is dominated by `Gasoline` cars, with `Hybrid`s three orders of magnitude smaller. We could drop the non-`Gasoline` options from here.
2. `Comfort`, `Performance`, `Exterior Styling`, `Interior Design` and `Reliability` have 3-, 4- and 5-star rating entries; if we consider a poor rating to be lower than 2.5 stars, then we only have better than average (2.5 stars) rating cars for all these categories and we don't necessarily have a clear choice to filter these columns
3. There are more listings in `TX` than `WA`
4. `make`, `seller_rating` and `consumer_stars` are high-cardinality features

We'll look at the influence of each of these categorical features on the main numerical feature (`MPG`, which visually showed the strongest linear trend with the target)

In [ ]:
plot_single_feature_target(
    df_train_filtered,
    "MPG",
    "price",
    ["MPG", "price", "State", "make", "year"],
    cats_cols_bar_chart,
    12,
    12,
    60,
    1,
    0.5,
    "category20",
    title_horizontal_offset=45,
    title_vertical_offset=-3,
    fig_size=(250, 300),
)

**Observations**
1. When applying the second filter to the `MPG` column, say at 80 MPG, to remove the outlying points from the linear grouping, we would notably give up
   - all `Hybrid`s and `Electric` (`Fuel Type`) vehicles
     - these splot distinctly separate from `Gasoline`
   - some `1` (single)-speed transmission cars

This filter is required into to improve the linear relationship between `MPG` and the target `price` column in the data, since none of the other scraped numerical features have even a visible linear trend which is a requirement for a linear regression model.

<a id="pre-processing"></a>

## 5. [Pre-processing](#pre-processing)

We'll create a pipeline object with custom transformers to transform (filter, change dtypes, one-hot encode) the data

In [ ]:
pipe_transformer = Pipeline(
    steps=[
        # ("duplicates", DFDropDupicates(keep="first")),
        ("filter_mpg", DFFilterNumerical("MPG", 80, "lt")),
        ("filter_mileage", DFFilterNumerical("Mileage", 10000, "lt")),
        ("nan", DFDropNa(cols=all_selected, how="any")),
        ("str_dtype", DFColTypeChanger(["consumer_stars", "seller_rating"], "str")),
        ("ohe", DFOneHotEncoder(cats, "=")),
    ]
)

We'll now use this pipeline to transform the training and testing data

In [ ]:
pipe_transformer.fit(X_train)
# print(pipe_transformer.transform(X_train).shape)
# print(pipe_transformer.transform(X_test).shape)
X_train = pipe_transformer.fit_transform(X_train)
X_test = pipe_transformer.transform(X_test)
y_train = y_train.loc[X_train.index]
y_test = y_test.loc[X_test.index]
print(X_train.shape)
print(X_train.shape, len(y_train))
print(X_test.shape, len(y_test))
display(X_train.head(2))
display(X_train.loc[:, X_train.columns.duplicated()].head(2))

In [ ]:
dummified_cols_flat = list(X_train.filter(regex="="))
print(len(dummified_cols_flat))

We'll summarize the transformed train and test splits below

In [ ]:
df_split_summary = pd.DataFrame.from_dict(
    {
        "train_min": y_train.min(),
        "train_max": y_train.max(),
        "test_min": y_test.min(),
        "test_max": y_test.max(),
        "train_var": y_train.var(),
        "test_var": y_test.var(),
    },
    orient="index",
).T
df_split_summary["var_diff"] = (
    (df_split_summary["test_var"] - df_split_summary["train_var"])
    / df_split_summary["train_var"]
) * 100
display(df_split_summary)

**Observations**
1. The train and test splits have a similar maximum target value and variance.
2. The minimum of the training set is approx. $4,000 lower than that for the test set. Without apriori knowledge of real (OOS) data, it may not possible to control this minimum value without adding some bias to the random splitting process. Given the size of the the dataset overall (nearly 3,000 observations overall), it is possible that the minimum value could change as the number of observations grows. For the current case, this splitting will be left as-is.

<a id="preparing-for-modeling-and-feature-selection"></a>

## 6. [Preparing for modeling and feature selection](#preparing-for-modeling-and-feature-selection)

In preparation for choosing a model and set of features, we'll define helper functions to help re-run several steps

In [ ]:
def model_cross_validator(
    pipe, model_name, num_feats_list, X, y, cols, scoring_metric="r2",
):
    """Perform KFCV for a single model or pipeline"""
    cv_score_scaler = -1 if type(scoring_metric) == "function" else 1
    d = {}
    cv_results = cross_validate(
        estimator=pipe,
        X=X,
        y=y,
        cv=RepeatedKFold(n_splits=5, n_repeats=5, random_state=1000),
        scoring=scoring_metric,
        return_train_score=True,
        return_estimator=True,
        n_jobs=-1,
    )
    # print(cv_results['train_score'], cv_results['test_score'])
    d["CV Train"] = cv_score_scaler * np.mean(cv_results["train_score"])
    d["CV Test"] = cv_score_scaler * np.mean(cv_results["test_score"])
    df_scores = pd.DataFrame.from_dict(d, orient="index").T
    # display(df_scores)
    model_obj = cv_results["estimator"][0].named_steps["reg"]
    if hasattr(model_obj, "coef_"):
        # print(model_obj)
        model_coefficients = model_obj.coef_
    else:
        model_coefficients = np.array([np.nan] * len(list(X)))
    df_coefs = get_model_coefs(
        coefs_array=model_coefficients, model_name=model_name, selected_cols=list(X),
    ).reset_index(drop=False)
    df_scores["model"] = model_name
    # print(cv_results["estimator"][0].named_steps["reg"].coef_)
    # display(cv_results["estimator"])
    return (
        df_scores,
        df_coefs,
        cv_results["estimator"][0].named_steps["reg"],
        cv_results,
    )

In [ ]:
def configuration_assesser(X, y, preprocessor, selected_cols, scoring_metric="r2"):
    """
    Perform KFCV on model(s) and return mean CV scores and model coefficients
    """
    df_scores = []
    df_coefs = []
    fitted_models = []
    cv_results_all = []
    for model in [
        linear_model.LinearRegression(),
        linear_model.LassoCV(alphas=[-100.0, 10, 50.0, 1500.0], cv=5, max_iter=1500),
        linear_model.RidgeCV(alphas=[-100.0, 10, 50.0, 1500.0], cv=5),
        linear_model.ElasticNetCV(
            l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
            alphas=[-100.0, 10, 50.0, 1500.0],
            cv=5,
        ),
        DummyRegressor(strategy="mean"),
        DummyRegressor(strategy="median"),
    ]:
        pipe = Pipeline(steps=[("preprocessor", preprocessor), ("reg", model)])
        model_name = (
            type(model).__name__
            if "Dummy" not in type(model).__name__
            else type(model).__name__ + f"_{model.strategy}"
        )
        df_cv_scores, df_coef_vals, model_fitted, cv_results = model_cross_validator(
            pipe=pipe,
            model_name=model_name,
            num_feats_list=nums,
            X=X,
            y=y,
            cols=selected_cols,
            scoring_metric=scoring_metric,
        )
        df_scores.append(df_cv_scores)
        df_coefs.append(df_coef_vals)
        fitted_models.append(model_fitted)
        cv_results_all.append(cv_results)
    df_sc = pd.concat(df_scores, axis=0).reset_index(drop=True)
    df_coefs_all = reduce(lambda x, y: pd.merge(x, y, on="index"), df_coefs)
    return df_sc, df_coefs_all, fitted_models, cv_results_all

We are now ready to pass several models into this pipeline and perform feature selection as well as select a model

<a id="base-model-numerical-features-only"></a>

## 7. [Base model - numerical features only](#base-model-numerical-features-only)

As a base model, we'll consider only the numerical features in the listings dataset

In [ ]:
selected_columns = nums
display(X_train[selected_columns].head(2))

<a id="raw-features"></a>

### 7.1. [Raw features](#raw-features)

We'll instantiate a pipeline to process (scale) the numerical features and perform no processing on the categorical features

We'll start by only using the numerical feature that showed the strongest linear relationship to the target during visual exploratory data analysis

In [ ]:
numericals_transformer = Pipeline(steps=[("scaler", StandardScaler()),])

preprocessor = ColumnTransformer(
    transformers=[("num", numericals_transformer, ["MPG"])], remainder="passthrough",
)

We'll run this feature through the helper function `configuration_assesser` to perform K-fold cross-validation. We'll check performance for three linear models.

In [ ]:
df_sc, df_coefs, m_fitted, cv_results_all = configuration_assesser(
    X=X_train[["MPG"]].copy(),
    y=y_train,
    preprocessor=preprocessor,
    selected_cols=selected_columns,
    scoring_metric=make_scorer(rmse_scorer, greater_is_better=False),
    # scoring_metric="r2",
)
display(df_sc)

Note that when `MPG` was replaced by `tank_volume`, the results found were

In [ ]:
pd.DataFrame.from_dict(
    {
        "CV Train": {
            0: 6481.154313256067,
            1: 6481.1620281921605,
            2: 6481.185298013487,
            3: 6481.1620281921605,
            4: 7598.918657028725,
            5: 7603.077576888589,
        },
        "CV Test": {
            0: 6484.1624923519885,
            1: 6484.1675242215315,
            2: 6484.183501694554,
            3: 6484.1675242215315,
            4: 7600.869861593557,
            5: 7606.112011843166,
        },
        "model": {
            0: "LinearRegression",
            1: "LassoCV",
            2: "RidgeCV",
            3: "ElasticNetCV",
            4: "DummyRegressor_mean",
            5: "DummyRegressor_median",
        },
    },
    orient="index",
).T

Since the `RMSE`s, for all models, were higher than with `MPG` alone, and due to the correlation between `MPG` and `tank_volume` (approx. -0.68 `R^2`), which would preclude interpretability of linear model coefficients, further analysis in this notebook will only use `MPG` and leaves out `tank_volume`.

Now, we'll use all numerical features

In [ ]:
numericals_transformer = Pipeline(steps=[("scaler", StandardScaler()),])

preprocessor = ColumnTransformer(
    transformers=[("num", numericals_transformer, nums)], remainder="passthrough",
)

In [ ]:
df_sc, df_coefs, m_fitted, cv_results_all = configuration_assesser(
    X=X_train[selected_columns].copy(),
    y=y_train,
    preprocessor=preprocessor,
    selected_cols=selected_columns,
    scoring_metric=make_scorer(rmse_scorer, greater_is_better=False),
    # scoring_metric="r2",
)
display(df_sc)

**Observations**
1. The low scores across in- and out-of-sample splits suggests we should add complexity by including more features. Since we saw some of the numerical features did not show perfectly normal distributions, this could influenced by the distribution of the input features - transformations to improve the distribution of these numerical features will be investigated next.
2. Train and test scores are similar to eachother so, at the very least, the models are not overfitting to the training data without being able to generalize to OOS data.
3. As mentioned in the `README.md` and above, the objective of building this model is to predict car prices to within \$5,000 of the true price
   - if we consider RMSE as our metric then, while this model is doing better than baseline models (taking the median or mean values), the desired level of performance is missed by approximately \$1,000
4. There is evidence of high bias (lacking complexity in features or type of model used) and low variance (difference between training and testing scores is small).
5. Comparing the scores, for all models, relative to the single `MPG` feature (which earlier showed the strongest linear relationship to the target), we can see that the additional numerical features do offer a small benefit in terms of improving the cross-validation scores. For this reason, we'll retain those features in subsequent analysis.

<a id="transformed-features"></a>

### 7.2. [Transformed features](#transformed-features)

Before moving on from the numerical features, we'll investigate the influence of the following feature transformations
- Quantile ([1](https://www.hydrol-earth-syst-sci.net/16/1085/2012/), [2](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2921808/))
- Power (Yeo-Johnson) ([1](https://en.wikipedia.org/wiki/Power_transform#Yeo%E2%80%93Johnson_transformation), [2](https://www.stat.umn.edu/arc/yjpower.pdf))
- `log(1 + x)`

<a id="quantile-transformation"></a>

#### 7.2.1. [Quantile transformation](#quantile-transformation)

First, we'll apply a quantile transformation to the input features.

We'll define a quantile transformer object below

In [ ]:
numericals_transformer = Pipeline(
    steps=[
        (
            "quantiles",
            QuantileTransformer(n_quantiles=len(X_train), output_distribution="normal"),
        ),
        ("scaler", StandardScaler()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[("num", numericals_transformer, nums)], remainder="passthrough",
)

We'll transform the features and the target, using this transformer object, and generate a pair plot showing the relationship between the
- (top row) transformed features and the raw target
- (bottom row) transformed features and the transformed target

In [ ]:
tr = QuantileTransformer(output_distribution="normal")
y_trans = tr.fit_transform(y_train.to_frame())
cols = list(set(nums) - set(["MPG"]))
if type(tr).__name__ != "FunctionTransformer":
    y_trans = pd.Series(y_trans.flatten(), index=y_train.index, name=y_train.name)

X_train_trans = pd.concat(
    [
        pd.DataFrame(
            Pipeline(
                steps=[("quantiles", numericals_transformer.named_steps["quantiles"])]
            ).fit_transform(X_train[cols]),
            index=X_train.index,
        ),
        X_train["MPG"],
        y_trans,
    ],
    axis=1,
)
X_train_trans.columns = cols + ["MPG"] + ["price_trans"]
plot_pairplot(
    df=pd.concat([X_train_trans, y_train], axis=1),
    yvar=["price", "price_trans"],
    vars_to_plot=list(X_train_trans),
    color_by_col=None,
    plot_specs=None,
    wspace=0.05,
    hspace=0.05,
    fig_size=(18, 6),
)

In [ ]:
plot_multiple_histograms(
    X=X_train,
    cols_to_plot=list(set(selected_columns) - set(["MPG"])),
    X_trans=X_train_trans,
    hspace=0.2,
    wspace=0.2,
    alpha=0.95,
    fig_size=(24, 6),
)

**Observations**
1. (From the histograms), the distributions have improved but (from the two rows of scatter plots above the histograms) the features still do not display a linear trend with the raw or transformed version of the target.

Now, we'll assess the models

In [ ]:
df_sc, df_coefs, m_fitted, cv_results_all = configuration_assesser(
    X=X_train[selected_columns],
    y=y_train,
    preprocessor=preprocessor,
    selected_cols=selected_columns,
    scoring_metric=make_scorer(rmse_scorer, greater_is_better=False),
    # scoring_metric="r2",
)
display(df_sc)

**Observations**
1. The train and validation scores are moving together, but are slightly lower than for the raw features.

We'll show the coefficients selected by the models, sorted by value of the coefficient

In [ ]:
df_stylers = []
for model_name in ["LinearRegression", "LassoCV", "RidgeCV"]:
    df_cs = pd.DataFrame(
        df_coefs.sort_values(by=model_name, ascending=False)[["index", model_name]]
    )
    df_styler = df_cs.style.set_table_attributes("style='display:inline'").set_caption(
        model_name
    )
    df_stylers.append(df_styler)

display_html(
    df_stylers[0]._repr_html_()
    + df_stylers[1]._repr_html_()
    + df_stylers[2]._repr_html_(),
    raw=True,
)

We'll plot the cross-validation scores below. If coefficients vary significantly when changing the input dataset their robustness is not guaranteed, and they should probably be interpreted with caution.

In [ ]:
plot_cv_scores(cv_results_all[0], selected_columns, fig_size=(8, 4))

**Observations**
1. From the reasonably constrained error bars, it is reassuring that none of the model's coefficients vary significantly when varying the input data, as was done during cross-validation.
2. It is clear that the `MPG` related features are prominent.
3. It appears that *cars.com* customer reviews are the next most important numerical factor to users of the site but this is approx. X5 weaker than the miles per gallon for the vehicle.

<a id="yeo-johnson-transformation"></a>

#### 7.2.2. [Yeo-Johnson transformation](#yeo-johnson-transformation)

Next, we'll apply a quantile transformation to the input features.

We'll define a power (yeo-johnson) transformer object below

In [ ]:
numericals_transformer = Pipeline(
    steps=[
        ("power", PowerTransformer(method="yeo-johnson")),
        ("scaler", StandardScaler()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[("num", numericals_transformer, nums)], remainder="passthrough",
)

We'll transform the features and generate a pair plot

In [ ]:
tr = PowerTransformer(method="yeo-johnson")
y_trans = tr.fit_transform(y_train.to_frame())
cols = list(set(nums) - set(["MPG"]))
if type(tr).__name__ != "FunctionTransformer":
    y_trans = pd.Series(y_trans.flatten(), index=y_train.index, name=y_train.name)

X_train_trans = pd.concat(
    [
        pd.DataFrame(
            Pipeline(
                steps=[("power", numericals_transformer.named_steps["power"])]
            ).fit_transform(X_train[cols]),
            index=X_train.index,
        ),
        X_train["MPG"],
        y_trans,
    ],
    axis=1,
)
X_train_trans.columns = cols + ["MPG"] + ["price_trans"]
plot_pairplot(
    df=pd.concat([X_train_trans, y_train], axis=1),
    yvar=["price", "price_trans"],
    vars_to_plot=list(X_train_trans),
    color_by_col=None,
    plot_specs=None,
    wspace=0.05,
    hspace=0.05,
    fig_size=(18, 6),
)

**Observations**
1. While the scales have changed, the relationship between each of the transformed features and the target column remains the same
   - the same observation is true for their relationship with the transformed target column

In [ ]:
plot_multiple_histograms(
    X=X_train,
    cols_to_plot=list(set(selected_columns) - set(["MPG"])),
    X_trans=X_train_trans,
    hspace=0.2,
    wspace=0.2,
    alpha=0.95,
    fig_size=(24, 6),
)

**Observations**
1. Distributions have improved but not all closely resemble a normal distribution.
2. The 95% c.i.'s are shown as shaded green regions, based on the assumption that the distribution is normal and peaks at a mean (central) value, but these are unreliable as the transformed (and raw) distributions are not normal.

Now, we'll assess the models

In [ ]:
df_sc, df_coefs, m_fitted, cv_results_all = configuration_assesser(
    X=X_train[selected_columns],
    y=y_train,
    preprocessor=preprocessor,
    selected_cols=selected_columns,
    scoring_metric=make_scorer(rmse_scorer, greater_is_better=False),
    # scoring_metric="r2",
)
display(df_sc)

**Observations**
1. The train and validation scores approximately the same as those for the raw features.

As befoire, we'll show the coefficients selected by the models, sorted by value of the coefficient

In [ ]:
df_stylers = []
for model_name in ["LinearRegression", "LassoCV", "RidgeCV"]:
    df_cs = pd.DataFrame(
        df_coefs.sort_values(by=model_name, ascending=False)[["index", model_name]]
    )
    df_styler = df_cs.style.set_table_attributes("style='display:inline'").set_caption(
        model_name
    )
    df_stylers.append(df_styler)

display_html(
    df_stylers[0]._repr_html_()
    + df_stylers[1]._repr_html_()
    + df_stylers[2]._repr_html_(),
    raw=True,
)

Again, `MPG` related features are prominent.

In [ ]:
plot_cv_scores(cv_results_all[0], selected_columns, fig_size=(8, 4))

<a id="log1plus-transformation"></a>

#### 7.2.3. [`log1plus` transformation](#log1plus-transformation)

Finally, we'll apply a `log1plus` transformation to the input features.

We'll define a `log1plus` function transformer object below

In [ ]:
numericals_transformer = Pipeline(
    steps=[
        ("log1p", FunctionTransformer(np.log1p, inverse_func=np.expm1)),
        ("scaler", StandardScaler()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[("num", numericals_transformer, nums)], remainder="passthrough",
)

In [ ]:
cols = list(set(nums) - set(["MPG"]))
X_train_trans = pd.concat(
    [
        pd.DataFrame(
            Pipeline(
                steps=[("log1p", numericals_transformer.named_steps["log1p"])]
            ).fit_transform(X_train[cols]),
            index=X_train.index,
        ),
        X_train["MPG"],
        y_trans,
    ],
    axis=1,
)
X_train_trans.columns = cols + ["MPG"] + ["price_trans"]
plot_pairplot(
    df=pd.concat([X_train_trans, y_train], axis=1),
    yvar=["price", "price_trans"],
    vars_to_plot=list(X_train_trans),
    color_by_col=None,
    plot_specs=None,
    wspace=0.05,
    hspace=0.05,
    fig_size=(18, 6),
)

In [ ]:
plot_multiple_histograms(
    X=X_train,
    cols_to_plot=list(set(nums) - set(["MPG"])),
    X_trans=X_train_trans,
    hspace=0.2,
    wspace=0.2,
    alpha=0.95,
    fig_size=(18, 6),
)

**Observations**
1. Distributions appear similar to the Yeo-Johnson transformed distributions.
2. Again, shaded regions can be ignored since the distributions are not normal.

Now, we'll assess the models

In [ ]:
df_sc, df_coefs, m_fitted, cv_results_all = configuration_assesser(
    X=X_train[selected_columns],
    y=y_train,
    preprocessor=preprocessor,
    selected_cols=selected_columns,
    scoring_metric=make_scorer(rmse_scorer, greater_is_better=False),
    # scoring_metric="r2",
)
display(df_sc)

We'll show the coefficients selected by the models, sorted by value of the coefficient

In [ ]:
df_stylers = []
for model_name in ["LinearRegression", "LassoCV", "RidgeCV"]:
    df_cs = pd.DataFrame(
        df_coefs.sort_values(by=model_name, ascending=False)[["index", model_name]]
    )
    df_styler = df_cs.style.set_table_attributes("style='display:inline'").set_caption(
        model_name
    )
    df_stylers.append(df_styler)

display_html(
    df_stylers[0]._repr_html_()
    + df_stylers[1]._repr_html_()
    + df_stylers[2]._repr_html_(),
    raw=True,
)

In [ ]:
plot_cv_scores(cv_results_all[0], selected_columns, fig_size=(8, 4))

**Observations**
1. The train and validation scores approximately the same as those for the raw features.
2. As with all transformed and raw features, train and validation scores move together.
3. Coefficients do not vary significantly as the training data is changed
   - Again, the `MPG` related feature is prominent

**Discussion**
1. While the quantile transformation best improved the shape of the distributions of the numerical features, the best compromise between shape of distribution and scores came from using the Yeo-Johnson transformation. This should be used in subsequent analysis.
2. For all three transformations applied to the target `price` variable, very similar cross-validation scores were returned compared to those on untransformed data for `LinearRegression` and `RidgeCV`. `LassoCV` and `ElasticNetCV` (programmed as `LassoCV`) returned (unrealistic) zero error scores for the transformed version of the target. As the bimodal effect was weak in the raw data, the untransformed target data will be used in futher analysis.

<a id="polynomial-and-interaction-terms"></a>

### 7.3. [Polynomial and Interaction terms](#polynomial-and-interaction-terms)

We'll now explore whether non-linearities and inter-feature interactions can result in a linear relationship with the target

We'll define a ppolynomial feature transformer object below, that will also generate feature interactions

In [ ]:
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
X_train_poly = pd.DataFrame(
    poly.fit_transform(X_train[selected_columns]),
    columns=poly.get_feature_names(),
    index=X_train.index,
)

We'll now programmatically assemble names of the newly generated features

In [ ]:
def get_polynomial_interaction_feats_names(
    df: pd.DataFrame, source_column_names: list
) -> list:
    d = {f"x{k}": num_col_name for k, num_col_name in enumerate(source_column_names)}
    new_feature_names = []
    for c in list(df):
        # print(c)
        if " " not in c:
            new_s = c.replace(c.split("^")[0], d[c.split("^")[0]])
            # print(c, c.split("^")[0], new_s)
        else:
            new_s = c.replace(c.split(" ")[0], d[c.split(" ")[0]]).replace(
                c.split(" ")[1], d[c.split(" ")[1]]
            )
            # print(c, c.split(" ")[0], c.split(" ")[1], new_s)
        new_feature_names.append(new_s)
    return new_feature_names

In [ ]:
X_train_poly.columns = get_polynomial_interaction_feats_names(X_train_poly, nums)

Finally, we'll instantiate a new preprocessor object where we'll specify the few feature names

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline(steps=[("scaler", StandardScaler())]), list(X_train_poly))
    ],
    remainder="passthrough",
)

In [ ]:
plot_pairplot(
    df=pd.concat([X_train_poly, y_train], axis=1),
    yvar=["price"],
    vars_to_plot=list(X_train_poly),
    color_by_col=None,
    plot_specs=None,
    wspace=0.05,
    hspace=0.15,
    fig_size=(48, 4),
)

**Observations**
1. With the exception of `MPG^2`, the simple non-linearities and feature interactions generated do not introduce new features with a linear relationship to the target.

A further problem with this is that the new features are correlated to eachother and to some of the existing fetures, as shown from a correlation heatmap below

In [ ]:
plot_corr_map(
    df=pd.concat([X_train_poly, y_train], axis=1),
    annot=False,
    annot_fmt=None,
    fig_size=(12, 8),
)

**Discussion**
1. Due to the problem of [non-moderate multi-collinearity](https://statisticsbyjim.com/regression/multicollinearity-in-regression-analysis/) introduced by adding polynomial and interaction terms, we'll ignore this type of feature engineering and exclude non-linear and interaction terms from the data. The disadvantage is that this will lower the predictive power of the scraped numerical features, since these complex combinations will not be included, but it will maintain model coefficient interpretability which is one of the benefits of linear models.

**Duke (i):** multi-collinearity must not be observed between input variables/features.

Due to this requirement, we cannot include pairs of features that are highly correlated (high `R^2`, showing up as deep red or green on this heatmap of correlation coefficients); instead, we have to choose one and drop the other feature in such pairs. This rules out the features generated due to interaction terms and ploynomials. Alternatively, we could have retained these and been forced to use `LassoCV`, which would have zeroed out multi-collinear features.

<a id="include-categorical-features"></a>

## 8. [Include categorical features](#include-categorical-features)

<a id="best-categorical-features"></a>

### 8.1. [Best categorical features](#best-categorical-features)

Next, we'll show the best set of categorical features found from iteratively adding one feature at a time and checking the cross-validation scores which are shown further down in [sub-section 8.2](#summary-of-adding-categorical-features)

In [ ]:
selected_columns = nums + dummified_cols_flat
display(X_train[selected_columns].head(1))

We'll again run these through the helper function in order to report on multiple models simultaneously

In [ ]:
numericals_transformer = Pipeline(
    steps=[
        ("power", PowerTransformer(method="yeo-johnson")),
        ("scaler", StandardScaler()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[("num", numericals_transformer, nums)], remainder="passthrough",
)

In [ ]:
df_sc, df_coefs, m_fitted, cv_results_all = configuration_assesser(
    X=X_train[selected_columns].copy(),
    y=y_train,
    preprocessor=preprocessor,
    selected_cols=selected_columns,
    scoring_metric=make_scorer(rmse_scorer, greater_is_better=False),
    # scoring_metric="r2",
)
display(df_sc)

**Observations**
1. KFCV scores improved suggesting the categorical features were helpful.
2. In- and out-of-sample scores are moving together so regularization is not the top priority.
3. The bias has reduced as the training scores have improved relative to only using the numerical features.
4. The variance is relatively similar as, again, the training and testing scores are moving together and there is a small difference between them

We'll show the model coefficients below

In [ ]:
df_stylers = []
df_coeffs = []
for model_name in ["LinearRegression", "LassoCV", "RidgeCV", "ElasticNetCV"]:
    df_cs = pd.DataFrame(
        df_coefs.sort_values(by=model_name, ascending=False)[["index", model_name]]
    )
    df_cs.rename(columns={"index": "feature"}, inplace=True)
    df_styler = df_cs.style.set_table_attributes("style='display:inline'").set_caption(
        model_name
    )
    df_stylers.append(df_styler)
    df_coeffs.append(df_cs)

display_html(
    df_stylers[0]._repr_html_()
    + df_stylers[1]._repr_html_()
    + df_stylers[2]._repr_html_()
    + df_stylers[3]._repr_html_(),
    raw=True,
)

We'll also print the amount of penalization chosen by cross validation for each of the regularized models

In [ ]:
d_alphas = {}
for model_obj in m_fitted[1:]:
    mname = type(model_obj).__name__
    if "Dummy" not in mname:
        d_alphas[mname] = model_obj.alpha_
pd.DataFrame.from_dict(d_alphas, orient="index").rename(columns={0: "best_alpha"})

**Observations**
1. The scale of coefficients for the `RidgeCV` and `LinearRegression` are similar to eachother. Since `LassoCV` uses a different norm than `RidgeCV`, and we enforced this same norm in `ElasticNetCV` (from the `l1_ratio` hyper-parameter), both of these models (`LassoCV` and `ElasticNetCV`) are similar to eachother. Unlike regularization with `Lasso`, `Ridge` does not zero-out features; instead the weight is shared between the two or more predictive variables. Some features zeroed out by `LassoCV` carry strong weight (of the zeroed out features) in the `RidgeCV` coefficients. For `LassoCV`, this zeroed weight is distributed among the other features, excluding `miles-per-gallon`.
2. The models are not over-fitting to the training data. We reach this conclusion from looking at the mean train (`CV Train`) and validation (`CV Test`) split scores from cross-validation. The main use of regularization is to counter over-fitting, which is evidently not a problem here as the test scores are very similar to the training scores. With strong regularization (the best chosen `alpha` values for `RidgeCV`, `ElasticNetCV` and `LasoCV` was 10), the coefficients of all the regularized models lie in a similar min-max range. However, the range of coefficients is larger for `LinearRegression`, which could be caused by multi-collinearity between combinations of features ([1](https://stats.stackexchange.com/a/421511/144450), [2](https://stats.stackexchange.com/a/64218/144450)) especially as high-cardinality categorical features are being added. This suggests regularization might be beneficial to control these coefficient values from blowing up and ensure their interpretability later on. For this reason, based on scores with this best set of categorical features and the earlier chosen numerical ones, choosing a regularized model could be beneficial.

<a id="summary-of-adding-categorical-features"></a>

### 8.2. [Summary of adding categorical features](#summary-of-adding-categorical-features)

The process of iteratively adding a single categorical feature, referred to [above](#best-categorical-features), to a base model comprising `Fuel Type`, `Drivetrain` and `State`, is reported below.

When `year` was added to the a base categorical feature set of `Fuel Type`, `Drivetrain` and `State`, the results found were

In [ ]:
pd.DataFrame.from_dict(
    {
        "CV Train": {
            0: 5473.387724126905,
            1: 5483.134595981816,
            2: 5488.360808508998,
            3: 5483.134595981816,
            4: 7600.754921755837,
            5: 7604.997140152696,
        },
        "CV Test": {
            0: 5512.5436290325315,
            1: 5512.969239744665,
            2: 5518.157019603685,
            3: 5512.969239744665,
            4: 7601.565006474694,
            5: 7605.959439060718,
        },
        "model": {
            0: "LinearRegression",
            1: "LassoCV",
            2: "RidgeCV",
            3: "ElasticNetCV",
            4: "DummyRegressor_mean",
            5: "DummyRegressor_median",
        },
    },
    orient="index",
).T

When `year` and `trans_speed` were added

In [ ]:
pd.DataFrame.from_dict(
    {
        "CV Train": {
            0: 5102.3929433172125,
            1: 5120.955154363325,
            2: 5134.480227590843,
            3: 5120.955154363325,
            4: 7598.728897140117,
            5: 7602.975035670904,
        },
        "CV Test": {
            0: 5170.96850526448,
            1: 5183.600486912249,
            2: 5196.18544570807,
            3: 5183.600486912249,
            4: 7600.587716921647,
            5: 7607.208725314137,
        },
        "model": {
            0: "LinearRegression",
            1: "LassoCV",
            2: "RidgeCV",
            3: "ElasticNetCV",
            4: "DummyRegressor_mean",
            5: "DummyRegressor_median",
        },
    },
    orient="index",
).T

With `year`, `trans_speed` and `seller_rating`, the following are the scores and the `LinearRegression` model returned on one-hot encoded `seller_rating` feature with a fitted coefficient that was approx. x3 larger than other features and very large `RMSE`

In [ ]:
pd.DataFrame.from_dict(
    {
        "CV Train": {
            0: 4800.603630049307,
            1: 4847.639393495595,
            2: 4854.909357773905,
            3: 4847.639393495595,
            4: 7598.728897140117,
            5: 7602.975035670904,
        },
        "CV Test": {
            0: 11925870424226.977,
            1: 4948.559967924518,
            2: 4954.095539079443,
            3: 4948.559967924518,
            4: 7600.587716921647,
            5: 7607.208725314137,
        },
        "model": {
            0: "LinearRegression",
            1: "LassoCV",
            2: "RidgeCV",
            3: "ElasticNetCV",
            4: "DummyRegressor_mean",
            5: "DummyRegressor_median",
        },
    },
    orient="index",
).T

Similar observations, of exploding validation set cross-validation scores and higher coefficients for a very small subset of features, were found when `consumer_stars` was added to `year` and `trans_speed`. The scores are shown below

In [ ]:
pd.DataFrame.from_dict(
    {
        "CV Train": {
            0: 4827.866967593768,
            1: 4865.630116619575,
            2: 4875.3716286546705,
            3: 4865.630116619575,
            4: 7598.728897140117,
            5: 7602.975035670904,
        },
        "CV Test": {
            0: 332152587630.0533,
            1: 4959.278431865635,
            2: 4964.551866565196,
            3: 4959.278431865635,
            4: 7600.587716921647,
            5: 7607.208725314137,
        },
        "model": {
            0: "LinearRegression",
            1: "LassoCV",
            2: "RidgeCV",
            3: "ElasticNetCV",
            4: "DummyRegressor_mean",
            5: "DummyRegressor_median",
        },
    },
    orient="index",
).T

When `year`, `trans_speed` and `Comfort` were added, scores and coefficients were better behaved

In [ ]:
pd.DataFrame.from_dict(
    {
        "CV Train": {
            0: 4956.976841898949,
            1: 4977.043027336144,
            2: 4986.760101298544,
            3: 4977.043027336144,
            4: 7598.728897140117,
            5: 7602.975035670904,
        },
        "CV Test": {
            0: 5027.286938265067,
            1: 5041.543322429582,
            2: 5049.651705377978,
            3: 5041.543322429582,
            4: 7600.587716921647,
            5: 7607.208725314137,
        },
        "model": {
            0: "LinearRegression",
            1: "LassoCV",
            2: "RidgeCV",
            3: "ElasticNetCV",
            4: "DummyRegressor_mean",
            5: "DummyRegressor_median",
        },
    },
    orient="index",
).T

With `year`, `trans_speed`, `Comfort` and `Performance`

In [ ]:
pd.DataFrame.from_dict(
    {
        "CV Train": {
            0: 4953.945333192869,
            1: 4975.043344549585,
            2: 4983.96203318336,
            3: 4975.043344549585,
            4: 7598.728897140117,
            5: 7602.975035670904,
        },
        "CV Test": {
            0: 5039.172005801402,
            1: 5043.034545328092,
            2: 5053.17241479394,
            3: 5043.034545328092,
            4: 7600.587716921647,
            5: 7607.208725314137,
        },
        "model": {
            0: "LinearRegression",
            1: "LassoCV",
            2: "RidgeCV",
            3: "ElasticNetCV",
            4: "DummyRegressor_mean",
            5: "DummyRegressor_median",
        },
    },
    orient="index",
).T

With `year`, `trans_speed`, `Comfort` and `Exterior Styling`

In [ ]:
pd.DataFrame.from_dict(
    {
        "CV Train": {
            0: 4941.72801416282,
            1: 4961.553376968924,
            2: 4970.584250384024,
            3: 4961.553376968924,
            4: 7598.728897140117,
            5: 7602.975035670904,
        },
        "CV Test": {
            0: 5015.9265156533775,
            1: 5029.440826583011,
            2: 5036.944482024336,
            3: 5029.440826583011,
            4: 7600.587716921647,
            5: 7607.208725314137,
        },
        "model": {
            0: "LinearRegression",
            1: "LassoCV",
            2: "RidgeCV",
            3: "ElasticNetCV",
            4: "DummyRegressor_mean",
            5: "DummyRegressor_median",
        },
    },
    orient="index",
).T

With `year`, `trans_speed`, `Comfort`, `Exterior Styling` and `Interior_Design`

In [ ]:
pd.DataFrame.from_dict(
    {
        "CV Train": {
            0: 4849.112335746027,
            1: 4870.507769797931,
            2: 4877.353421220105,
            3: 4870.507769797931,
            4: 7598.728897140117,
            5: 7602.975035670904,
        },
        "CV Test": {
            0: 4927.120360636142,
            1: 4938.4591029994,
            2: 4946.060803431385,
            3: 4938.4591029994,
            4: 7600.587716921647,
            5: 7607.208725314137,
        },
        "model": {
            0: "LinearRegression",
            1: "LassoCV",
            2: "RidgeCV",
            3: "ElasticNetCV",
            4: "DummyRegressor_mean",
            5: "DummyRegressor_median",
        },
    },
    orient="index",
).T

Finally, with `year`, `trans_speed`, `Comfort`, `Exterior Styling`, `Interior_Design` and `Reliability`

In [ ]:
pd.DataFrame.from_dict(
    {
        "CV Train": {
            0: 4829.946163513605,
            1: 4869.310940006296,
            2: 4872.917805000394,
            3: 4869.310940006296,
            4: 7598.728897140117,
            5: 7602.975035670904,
        },
        "CV Test": {
            0: 4914.357334726612,
            1: 4940.607277684173,
            2: 4947.6334368637135,
            3: 4940.607277684173,
            4: 7600.587716921647,
            5: 7607.208725314137,
        },
        "model": {
            0: "LinearRegression",
            1: "LassoCV",
            2: "RidgeCV",
            3: "ElasticNetCV",
            4: "DummyRegressor_mean",
            5: "DummyRegressor_median",
        },
    },
    orient="index",
).T

**Observations**
1. As the out-of-sample scores are not larger than the in-sample ones, there does not appear to be evidence of overfitting.
2. Consistent improvement in `CV Test` (validation split) scores, across all models, for `year`, `trans_speed`, `Comfort`, `Exterior Styling` and `Interior_Design`
2. When `Reliability` was added, although `LinearRegression` reported slightly lower scores, the other models were higher than when `Reliability` was not included, so this feature was not retained.

**Discussion**

As mentioned earlier, a regularized model will be chosen as all models are similar in scores, but coefficients seem slightly bettre behaved for those that were penalized. `RidgeCV` was selected, but subsequent sections showed similarity between results with `RidgeCV` and `LassoCV`.

<a id="assessing-best-model"></a>

## 9. [Assessing best model](#assessing-best-model)

In this section, we'll assess performane of the best regressor, which we decided to be `RidgeCV()` based on the cross-validation runs and the resulting range of model coefficients.

<a id="extracting-best-model"></a>

### 9.1. [Extracting best model](#extracting-best-model)

First, we'll instantiate a new model object.

Here, the model is extracted from the list of returned fitted models from cross-validation, but this model will be re-fit on all the training data.

In [ ]:
model_row = 2  # 2 since RidgeCV was third among the list of models tried

In [ ]:
best_model = df_sc.loc[model_row, "model"]
print(best_model)

Next, we'll instantiate a pipeline use the newly instantiated best model object to it, after the Yeo-Johnson feature transformer preprocessing step that was determined from cross-validation earlier to be the best for transforming the numerical features

In [ ]:
pipe = Pipeline(steps=[("preprocessor", preprocessor), ("reg", m_fitted[model_row])])

This pipeline object will be used in subsequent model evaluation steps.

<a id="get-best-model-coefficients"></a>

### 9.2. [Get best model coefficients](#get-best-model-coefficients)

First, we'll visualize the feature coefficients for the best model. Since this is a linear model, we can directly interpret the model'ls coefficients because we have
- scaled features
- removed multi-collinear features
- not observed wildly varying coefficient values during cross-validation earlier

In [ ]:
df_viz_best = (
    pd.DataFrame.from_dict(
        dict(zip(selected_columns, pipe.named_steps["reg"].coef_.tolist())),
        orient="index",
    )
    .reset_index()
    .rename(columns={"index": "feature", 0: best_model})
    .sort_values(by=best_model, ascending=False)
)
plot_coef_plot(xvar=best_model, df=df_viz_best, fig_size=(12, 12), save_fig=False)

**Observations**
1. The dominant features are related to `miles-per-gallon`, which is a numerical feature, and this is negative since its relationship to the target (listing price) was seen earlier to be negative.
   - other numerical features remain several times smaller than this feature
2. Dominant categorical features are `Transmission Speed`, `Drivetrain` and `fuelType` and they are stronger predictors than the other numerical features used.
3. It is notable that that `State` is not one of the strongest predictors. So, if it is included, it won't drive predictions significantly - if it was a significant predictor, we have been better off using a separate model per state.

<a id="residuals-for-predictions"></a>

### 9.3. [Residuals for predictions](#residuals-for-predictions)

We'll now plot the residual on the test set, using this object

In [ ]:
X_comp, pipe_final_fitted, fit_residual = plot_residual_manually(
    estimator=pipe,
    X_train=X_train[selected_columns],
    y_train=y_train,
    X_test=X_test[selected_columns],
    y_test=y_test,
)

**Observations**
1. A pattern is not clearly visible in the residuals. However, it does not resemble white noise due to the upper bound on vehicle prices that was set when retrieving the listings. This appears here as a 45-degree line, intersecting the x-axis at y=0 and at the x-value corresponding to the highest listed price in the test data.
2. A clear fan stricture increasing from `x=0` is not visible.
3. Since there is minor evidence of pattern in the plot, so some [heteroskedasticity (requires a distinctive fan or cone shape in residual plots)](https://statisticsbyjim.com/regression/heteroscedasticity-regression/) is likely present.

**Duke (i):** a linear **additive** relationship exists between the dependent and each independent variable.

and

**Duke (iv):** model residuals should display a normal distributed, centered around zero

Residuals are centered at a value of 0 as expected (the distribution will give a complimentary assessment of this later) and we don't see a pattern which would indicate systematic non-additive errors in the predictions.

**Duke (iii)**: errors terms show constant variance (no homoskedasticity)

As mentioned above, some qualitative evidence of heteroskedasticity is present due to the enforced upper bound of \$45,000 for listing price. The rule of thumb is OLS regression isn't too impacted by heteroscedasticity as long as the maximum variance is not greater than four times the minimum variance.

Here, due to the lower part of the residual plot being affected by the upper-bound cutoff of \$45,000, only the top half will be considered
- starting at the `Predicted Value` of \$20,000, the top half min-max range is approx. 5,000 USD
  - this is also true for the bottom half-range
- at the average (central) `Predicted Value` of \$30,000, the half-range is approx. 12,000 USD
  - this is also true for the bottom half-range
- ending at the `Predicted Value` of \$40,000, the upper half-range is approx. 14,000 USD
  - this is cannot be determined for the bottom half-range

As the min-max range is approx. 3, the best model here is not in violation of this rule of thumb. Also a large number of the residuals are less than these extreme values. These appear to suggest minimal heteroskedasticity.

**Duke ii**: Errors are statistically independent, meaning there is no correlation between consecutive errors

This cannot be tested here since the listing date was not scraped and this dataset is not a timeseries.

<a id="residual-distributions"></a>

#### 9.3.1. [Residual distributions](#residual-distributions)

We'll look at a histogram of the train and test splits, as well as their residuals

In [ ]:
y_pred = pipe_final_fitted.predict(X_test[selected_columns])
y_pred_train = pipe_final_fitted.predict(X_train[selected_columns])

In [ ]:
plot_multiple_histograms(
    X=pd.concat(
        [
            y_train.reset_index(drop=True).rename("train"),
            y_test.reset_index(drop=True).rename("test"),
        ],
        axis=1,
    ),
    cols_to_plot=["train", "test"],
    hspace=0.2,
    wspace=0.2,
    fig_size=(12, 6),
)

In [ ]:
plot_multiple_histograms(
    X=pd.concat(
        [
            (y_pred_train - y_train).rename("train residual").to_frame(),
            (y_pred - y_test).rename("test residual").to_frame(),
        ],
        axis=1,
    ),
    cols_to_plot=["train residual", "test residual"],
    hspace=0.2,
    wspace=0.2,
    fig_size=(12, 6),
)

**Observations**
1. As expected, the distributions have a right upper bound due to the maximum value enforced during retrieval of the listings.
2. Both distributions appear to be broadened Gaussian shapes, with a drop in the center around the average retrieved car price of $30,000.
3. The residuals are closer to normal distributions with a clear single peak, though both are also broadened in width shown by the lower-than-expected y-value where the green shaded 95 per-cent c.i. intersects the disitibution. Both are centered at zero, as expected.

These suggest if additional data were acquired at around the average value, the shape of the input data could be improved.

**Duke (iv): (contd)** model residuals should display a normal distributed, centered around zero

From the distributions, the residuals are centreed at zero but show an overall broadened normal distribution suggesting there are larger than ideal positive and negative residuals. The distribution of the test set is not perfectly symmetric; residuals of approx USD 4,000-5,000 and moreso USD6,000-8,000 are higher than expected for a normal distribution - the latter appears to be driving the skew more strongly however the former is of concern for the current project since it falls inside the error tolerance specified for predicting vehicle prices namely \$5,000. These sources of deviations from the normal shape of the distribution could intriduce bias into the model's predictions and we'll explore this further later in this analysis.

<a id="worst-performing-predictions"></a>

#### 9.3.2. [Worst performing predictions](#worst-performing-predictions)

Next, we'll show the top 10 highest residuals

In [ ]:
X_comp["pct_res"] = (X_comp["res"] / X_comp["test"]) * 100
display(
    X_comp.sort_values(by="res", ascending=False)[
        ["State=WA", "res", "test", "pct_res"]
    ].nlargest(20, ["res"])
)

In [ ]:
print(
    "Percent of predictions different from true price by more than $5,000: "
    f"{(len(X_comp.loc[X_comp['res'] > 5000]) / len(X_comp)) * 100:.1f}% "
    f"(out of {len(X_comp)} obs.)"
)

**Observations**
1. We can see that the largest residual (corresponding to the worst prediction) is approximately 12,500 dollars on a car whose price is approximately 27,000 dollars for a percent residual of approx. 46%.
2. The model's top 6 worst predictions contain five from the state of Washington, and all of these miss the true `price` by more than \$10,000.
4. Approximately thirteen percent of the predictions are different from the true prices by more than \$5,000.

<a id="residuals-separated-by-state"></a>

#### 9.3.3. [Residuals separated by State](#residuals-separated-by-state)

Finally, we'll check if the residuals show a trend based on the `State`

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x="pred", y="res", hue="State=WA", data=X_comp, ax=ax, s=80)
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Residuals")
ax.set_title("Model Residuals, by State")

**Observations**
1. As there aren't any clear patterns by `State`, this suggests using a single model, with `State` as a categorical feature, is acceptable for this data set. Creating a separate model for each state is likely not the main contributor to deviation in model predictions from the target.

<a id="quantile-quantile-plot"></a>

#### 9.3.4. [Quantile-quantile plot](#quantile-quantile-plot)

Next, we'll plot a QQ plot of the residual in order to check that the regression is linear in model parameters and correctly specified

In [ ]:
plot_qq(fit_residual)

**Duke (i) (contd)**: a linear additive relationship exists between the dependent and each independent variable.

We can check this by inspecting the QQ plot and require points in the central section to be very close to the 45-degree line.

The Q-Q (quantile-quantile) plot helps qualitatively determine if it is plausible that the data (residuals) come from a normal distribution. We can use the Normal Q-Q plot to check the assumption that the target data comes from a normal distribution. This plot is produced by graphing both sets of quantiles against one another. If both sets of quantiles have came from the same distribution, points should form a straight line. ([Link](https://data.library.virginia.edu/understanding-q-q-plots/)) 

The middle section of the QQ plot shown is very close to the diagonal red line, with evidence of light tailing (due to curvature at lower and higher quantiles). 

<a id="permutation-importances"></a>

### 9.4. [Permutation Importances](#permutation-importances)

Next, we'll look at the permutation importance of all the features that we've chosen. We'll use the version of the categorical data that has not been one-hot encoded and include the one-hot encoding step in a pipeline. This way the original categorical feature names, and not the dummified versions, will appear in this plot.

We'll manually apply the filtering needed - previously this was done inside a transformer pipeline that could also operate on target values. Now we'll do this separately on the train and test split since the pipeline will need to include a regression model at the end and so can't opreate on the target column.

In [ ]:
test_indices = X.loc[list(set(X.index.tolist()) - set(train_indices))].index
selected_columns_temp = nums + cats
X_train_filtered = X.loc[train_indices][
    (X.loc[train_indices]["MPG"] < 80) & (X.loc[train_indices]["Mileage"] < 10000)
][selected_columns_temp].dropna(subset=selected_columns_temp, how="any")
y_train_filtered = y.loc[X_train_filtered.index]
X_test_filtered = X.loc[test_indices][
    (X.loc[test_indices]["MPG"] < 80) & (X.loc[test_indices]["Mileage"] < 10000)
][selected_columns_temp].dropna(subset=selected_columns_temp, how="any")
y_test_filtered = y.loc[X_test_filtered.index]

We'll remove the filtering steps from the transformer pipeline and add the preprocessing and classification steps in - this will be the new pipeline that includes the regression model and can be used as an input into the `permutation_importances` module.

In [ ]:
pipe_transformer = Pipeline(
    steps=[
        # ("filter_mpg", DFFilterNumerical("MPG", 80, "lt")),
        # ("filter_mileage", DFFilterNumerical("Mileage", 10000, "lt")),
        ("nan", DFDropNa(cols=all_selected, how="any")),
        ("ohe", DFOneHotEncoder(cats, "=")),
        ("preprocessor", preprocessor),
        ("reg", m_fitted[model_row]),
    ]
)

We'll train the pipeline on training data and predict + score on OOS data

In [ ]:
pipe_transformer.fit(X_train_filtered, y_train_filtered)
y_test_pred = pipe_transformer.predict(X_test_filtered)
y_train_pred = pipe_transformer.predict(X_train_filtered)
display(
    pd.DataFrame.from_dict(
        {
            "train": r2_score(y_train_filtered, y_train_pred),
            "test": r2_score(y_test_filtered, y_test_pred),
        },
        orient="index",
    )
    .reset_index()
    .rename(columns={0: "r2", "index": "split"})
)

Having trained this pipeline and made predictions on the testing data, we'll now get the linear model (`RidgeCV`) coefficients and permutation importances

In [ ]:
df_viz_best = (
    pd.DataFrame.from_dict(
        dict(
            zip(
                selected_columns_temp,
                pipe_transformer.named_steps["reg"].coef_.tolist(),
            )
        ),
        orient="index",
    )
    .reset_index()
    .rename(columns={"index": "feature", 0: best_model})
)
df_viz_best = df_viz_best.reindex(
    df_viz_best[best_model].abs().sort_values(ascending=False).index
)
r = permutation_importance(
    pipe_transformer,
    X_train_filtered,
    y_train_filtered,
    scoring=make_scorer(rmse_scorer, greater_is_better=True),
    # scoring="r2",
    n_repeats=30,
    random_state=0,
    n_jobs=-1,
)
sorted_idx = r.importances_mean.argsort()
df_perm_imp = pd.DataFrame(r.importances[sorted_idx].T)
df_perm_imp.columns = X_test_filtered.columns[sorted_idx]
df_perm_imp = df_perm_imp.unstack(level=-1).reset_index()
df_perm_imp = df_perm_imp.rename(
    columns={"level_0": "feature", "level_1": "trial", 0: "importance"}
)
df_perm_imp = (
    df_perm_imp.set_index("feature")
    .loc[X_test_filtered.columns[sorted_idx][::-1]]
    .reset_index()
)

Graphing the linear model coefficients and permutation importances

In [ ]:
plot_side_by_side_bar_box(
    df_viz_best,
    best_model,
    df_perm_imp,
    X_train_filtered.columns[sorted_idx][::-1].tolist(),
    "darkred",
    "blue",
    25,
    "blue",
    12,
    12,
    115,
    -3,
    (350, 200),
)

**Observations**
1. The error bars show that the feature importances do not vary wildly as they are randomized so the influence of each feature on its own is reasonably well constrained with minimal influence of outliers (points lying outside the error bar range).
2. As with the linear model coefficients of the best model, *Miles-Per-Gallon* is the most important factor followed by *Drivetrain*. Permuting (randomizing) their respective orders (while holding all other factors constant) has the largest effect on the model's predictions of the OOS data, of all the factors that were investigated here.
3. Based on permutation importances, there are some features whose importance is relatively much weaker than the absolute values of their model coefficients suggest - `Comfort` and `year`. The opposite is true for `trans_speed`, which ranks higher in permutation importance than its coefficient suggests. Since permutation importance is free of problems in interpreting regularized model coefficients and correlated variables, it could be a better choice for the relative importance of these three features to the model. With the exception of these three features, there is general agreement between the relative positioning of the features on each of this plots, which is reassuring since it suggests errors in interpreting model coefficients is minimal.

<a id="prediction-error"></a>

### 9.5. [Prediction error](#prediction-error)

Next, we'll use the prediction error plot to show the actual targets from the dataset against the predicted values generated by our model. This will allow for a visualization of how much variance is in the model. We can use this diagnostic plot by comparing it against the 45 degree (slope of 1) line which corresponds to the prediction and model exactly matching eachother.

In [ ]:
show_yb_prediction_error(
    estimator=pipe_final_fitted, X=X_test[selected_columns], y=y_test,
)

**Duke (i): (contd.)** a linear additive relationship exists between the dependent and each independent variable.

We can check this from a plot of observed data vs predicted values for the test set - the predicted values should be symmetric around their line of best fit. While this is quanitatively the case from visual inspection and quantitatively from the `R^2` of 0.61, it is also clear the there is a difference between the best-possible linear fit and the one found here.i.e. there is room for improvement in the model.

**Observations**
1. The true points (grey dashed line) are asymetric relative to the prediction best fit line (black dashed line), attributed to the sub-optimal `R^2` value from the fitted model on the test set.

<a id="summary-of-scoring-metrics"></a>

### 9.6. [Summary of scoring metrics](#summary-of-scoring-metrics)

Next, we'll print scoring metrics on predictions made using the final pipeline (estimator)

These are shown first for the test set only

In [ ]:
df_metrics = get_test_metrics(
    metrics_wanted=["MAE", "MDAE", "RMSE", "MAXE", "MAPE", "R2", ".score()"],
    X_r=X_test[selected_columns],
    r=y_test,
    f=y_pred,
    est=pipe_final_fitted,
)
display(df_metrics)

Below are a subset of metrics, in addition to max and min values, for the training and testing sets

In [ ]:
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
rmse_test = mean_squared_error(y_test, y_pred, squared=False)
df_summ = (
    pd.DataFrame.from_dict(
        {
            "r2_train": r2_train,
            "r2_test": r2_test,
            "rmse_train": rmse_train,
            "rmse_test": rmse_test,
            "min_train": y_train.min(),
            "max_train": y_train.max(),
            "min_test": y_test.min(),
            "max_test": y_test.max(),
        },
        orient="index",
    )
    .rename(columns={0: "value"})
    .astype(float)
    .reset_index()
)
df_summ[["metric", "split"]] = df_summ["index"].str.split("_", expand=True)
df_summ[["value", "metric", "split"]].pivot(
    index="split", columns="metric", values="value"
)

**Observations**
1. As seen earlier, the training set has a lower minimum value than the testing set.
2. Both `R^2` and RMSE are better for the testing data than for the training data. A possible reason is inadequate splitting of the training and testing data.i.e. randomly setting 20% aside as the testing split may not be appropriate based on the nature of the dataset. Multiple train-test splits could be used to further explore this option and this will be done below.
3. Current model performance on the test set is just below (better than) the desired level for this project.i.e. the desired RMSE is \$5,000, which is just above the best model's preformance.

<a id="comparison-of-ordinary-linear-regression-between-sklearn-and-statsmodels"></a>

#### 9.6.1. [Comparison of ordinary linear regression between `sklearn` and `statsmodels`](#comparison-of-ordinary-linear-regression-between-sklearn-and-statsmodels)

Below is a brief comparison between non-penalized OLS regression using the `statsmodels` library and `LinearRegression()` from `sklearn`.

We can use the above filtered training and testing splits that were used in the [Permutation Importances](#permutation-importances) section, and pass this through the same transformer pipeline instantiated in that section, without the `sklearn` regressor object at the end, to transform the filtered training and testing data

In [ ]:
pipe_transformer = Pipeline(
    steps=[
        ("nan", DFDropNa(cols=all_selected, how="any")),
        ("ohe", DFOneHotEncoder(cats, "=")),
        ("preprocessor", preprocessor),
    ]
)
X_train_filtered_transformed = pd.DataFrame(
    pipe_transformer.fit_transform(X_train_filtered, y_train_filtered),
    columns=pipe_transformer.named_steps["ohe"].get_feature_names(),
    index=X_train_filtered.index,
)
X_test_filtered_transformed = pd.DataFrame(
    pipe_transformer.transform(X_test_filtered),
    columns=pipe_transformer.named_steps["ohe"].get_feature_names(),
    index=X_test_filtered.index,
)

<a id="using-statsmodels"></a>

##### **9.6.1.1. [Using `statsmodels`](#using-statsmodels)**

The OLS model is trained on the transformed training data and predictions are made ([1](https://stackoverflow.com/a/57816992/4057186), [2](https://www.statsmodels.org/stable/examples/notebooks/generated/predict.html)) on the transformed training and testing data

In [ ]:
olsres = sm.OLS(y_train_filtered, X_train_filtered_transformed).fit()
y_pred_train_sm = olsres.predict(X_train_filtered_transformed)
y_pred_test_sm = olsres.predict(X_test_filtered_transformed)

The resulting scoring metrics for predictions made using `statsmodels` OLS API are shown below, for train and test set predictions

In [ ]:
r2_train_sm = r2_score(y_train_filtered, y_pred_train_sm)
r2_test_sm = r2_score(y_test_filtered, y_pred_test_sm)
rmse_train_sm = mean_squared_error(y_train_filtered, y_pred_train_sm, squared=False)
rmse_test_sm = mean_squared_error(y_test_filtered, y_pred_test_sm, squared=False)
df_summ_sm = (
    pd.DataFrame.from_dict(
        {
            "r2_train": r2_train_sm,
            "r2_test": r2_test_sm,
            "rmse_train": rmse_train_sm,
            "rmse_test": rmse_test_sm,
        },
        orient="index",
    )
    .rename(columns={0: "value"})
    .astype(float)
    .reset_index()
)
df_summ_sm[["metric", "split"]] = df_summ_sm["index"].str.split("_", expand=True)
df_summ_sm[["value", "metric", "split"]].pivot(
    index="split", columns="metric", values="value"
)

<a id="using-sklearn"></a>

##### **9.6.1.1. [Using `sklearn`](#using-sklearn)**

By comparison, using `sklearn`'s `LinearRegression`

In [ ]:
lm = linear_model.LinearRegression()
lm.fit(X_train_filtered_transformed, y_train_filtered)
y_pred_train_sk = lm.predict(X_train_filtered_transformed)
y_pred_test_sk = lm.predict(X_test_filtered_transformed)

In [ ]:
r2_train_sk = r2_score(y_train_filtered, y_pred_train_sk)
r2_test_sk = r2_score(y_test_filtered, y_pred_test_sk)
rmse_train_sk = mean_squared_error(y_train_filtered, y_pred_train_sk, squared=False)
rmse_test_sk = mean_squared_error(y_test_filtered, y_pred_test_sk, squared=False)
df_summ_sk = (
    pd.DataFrame.from_dict(
        {
            "r2_train": r2_train_sk,
            "r2_test": r2_test_sk,
            "rmse_train": rmse_train_sk,
            "rmse_test": rmse_test_sk,
        },
        orient="index",
    )
    .rename(columns={0: "value"})
    .astype(float)
    .reset_index()
)
df_summ_sk[["metric", "split"]] = df_summ_sk["index"].str.split("_", expand=True)
df_summ_sk[["value", "metric", "split"]].pivot(
    index="split", columns="metric", values="value"
)

Results are very similar, as expected.

<a id="check-influence-of-choice-of-test-split"></a>

### 9.8. [Check influence of choice of test split](#check-influence-of-choice-of-test-split)

We'll now explore how the choice of splitting out the test set influences the model's performance. We'll do this using nested k-fold cross-validation on all (training + testing) data.

We'll do this with two plots
- (top) **fixed** train+validation and test sets in the outer cross-validation splits, with variable inner splits
  - this will be equivalent to using the test set from the train-test split that was created above
  - we would expect the train and test scores here to match those found from the best model and summarized in the table above
- (bottom) **varible** train+validation and test sets in outer cross-validation splits, with variable inner splits
  - here, the test split will be varied

In [ ]:
plot_nested_cv(
    pipe_final_fitted,
    pd.concat([X_train, X_test])[selected_columns],
    pd.concat([y_train, y_test]),
    NUM_NESTED_CV_TRIALS,
    PredefinedSplit(([-1] * len(y_train)) + [0] * len(y_test)),
    scorer=make_scorer(rmse_scorer, greater_is_better=False),
    # scorer="r2",
    outer_cv_n_splits=5,
    inner_cv_n_splits=5,
    hspace=0.05,
    wspace=0.05,
    fig_size=(12, 8),
)

**Notes**
1. Hyper-parameter optimization is not performed in the inner cross-validation; instead, the best found model is used as-is.
2. Average scores are shown as a solid line, with the shaded region showing the range of scores.
3. Due to the random nature of generating the test splits, the individual test split generated in the top plot is not guaranteed to re-occur in the bottom plot. For this reason, the exact test score that is returned when the test split is fixed does not appear when the test set is variable.

**Observations**
1. As expected, the top plot shows train and test scores (where the train - the train+validation set - and test set are always fixed, to match the single test-set splitting used earlier) that match those found above; as we can see, the training score is worse than the testing score.
2. From the bottom plot, depending on the choice of test split, the test score can be larger than the train score for this dataset. Across multiple test splits, this is not the case on average and the mean testing score is slightly worse than the mean training score.

**Discussion**

We have to think of test set generalization error as a random variable. The mean of the test distribution is the true generalization error for the test set. A distribution can be characterized by a mean and also a standard deviation. What makes one get a different test set sample is the draw of the data randomly returned.

We take samples from this distribution curve. Depending on the sample taken, it may or may not a good estimate of the mean of the distribution. We may get a good estimate of the generalization error. However, that is not always the case and the sample may be a poor estimate of the mean of the distribution. When a model is deployed to production, the sampled value for this error may also be significantly different from the mean just by chance.

Normal cross-validation using a train+validation split and test split, but does not account for the variance of the test set's error distribution due to the fact that we're taking a random test split. So, instead of using one test split, we could use several test splits to account for this variance. If we take multiple samples from the Gaussian, we would get a better estimate of the mean of the generalization error (which is the error of the test set).

For five test splits, we split the entire dataset into five possible test splits rather than just one. Now, each sample participates in the test split exactly once. We fit the same model five times and we get five test set errors, giving a mean test error and minimum and maximum of the test error. This (max - min) allows us to explore a range of OOS scores. It may be possible that one of these five test set scores is larger than the train+validatoin set score, but that may not be true for the other four test sets resulting in an average test set score than is infact lower than the train set score. This appears to be the case here, as seen from the solid red line (average test score) that is lower than the training+validation set score, even though the maximum test set score is higher than than the maximum train set score.

Small datasets, such as this one, are particularly susceptible to this problem. In this case, it is likely that the small size of the test set (approx. 500 observations) produces this large variability, which is reduced in the much larger (approx. X5) train+validation set where the influence of a small number of observations driving the increased test set variability is reduced. A better estimate of model performance in this case may be given by the averaged test set score (+/- its standard deviation) from nested cross-validation than a single test set score. Additionally, another round of filtering may be warranted to remove the influence of observations that are driving up the test set error.

<a id="check-of-bias-and-variance"></a>

### 9.8. [Check of bias and variance](#check-of-bias-and-variance)

Finally, we'll explore bias and variance related to this dataset

In [ ]:
prf = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("reg", RandomForestRegressor(max_leaf_nodes=350)),
    ]
)
show_yb_learning_curves(
    X=pd.concat([X_train[selected_columns], X_test[selected_columns]]),
    y=pd.concat([y_train, y_test]),
    pipes=[pipe, prf],
    hspace=0,
    wspace=0.1,
    n_splits=5,
    n_repeats=5,
    scorer=make_scorer(rmse_scorer, greater_is_better=False),
    fig_size=(16, 8),
)

**Observations and Discussion**
1. Both training and validation scores show a minor increase upto approx. 750 training observations. Beyond this point, the metric stays the same. This means adding additional training data won't noticeably improve the model performance. So, for this dataset, it may be better to consider
   - adding more features to the data, as this is very likely to increase complexity of the model and improve performance
   - switching to a non-linear regression model that can build a more complicated regressor from the existing data
     - from the learning curves for the `RandomForestRegressor`, we see that this is precisely the case
       - the training and validation scores are nearly doubled
       - these more complicated models generally require more data, and we can see this from the validation score which has not reached a plateau.i.e. it could benefit from more validation data.
2. The main indicator of a bias problem is a high validation error. The validation metric here, for the `RidgeCV` model, reaches a plateau at approx. \$5,000 which is right at our desired model performance indicated earlier.
   - For each listing, the model is off by this amount on average. So, the best linear model found here has a problem of moderate bias.
   - since the training error is also right at our desired performance, it means that the training data is also not fit well enough by the best linear model. This means the best model found here has moderate bias with respect to the training data.
3. A consistently narrow gap exists between the training and validation learning urves. This indicates low variance. The narrower the gap, the lower the variance for the following reasons
   - if the variance is high, then the model fits training data too well
     - when training data is fitted too well, the model will have trouble generalizing on data that hasn’t seen in training. When such an overfitting model is tested on its training set, and then on a validation set, the training error will be low and the validation error will generally be high. As we change training set sizes, this pattern continues, and the differences between training and validation errors will determine that gap between the two learning curves
     - the larger the difference between the two errors, the larger the variance
     - in this case, the gap is very narrow, indicating that the variance is low
   - high training RMSE scores also indicate low variance
     - if the variance is low, then the model will generate similar but over-simplified models as the length of the training observations is increased. Because the models are overly simplified, they do not fit the training data well enough, producing should a high training error.
4. Overall, the best linear model here suffers from a moderate bias and low variance problem. Adding more training observations is unlikely to improve this much. Two approaches to remedy this are
   - remove or decrease regularization
     - this will allow the model to better fit the training data and increase the variance as well as decrease the bias
       - in order to do this while preventing model coefficients from blowing up, as seen earlier for `LinearRegression`, some regularization is likely necessary for the current dataset
   - add more features
     - this requrires scraping more data or using a new set of features than the ones chosen at the start of this analysis (`v1`)
   - try a non-linear model
     - as seen from the learning curves for the `RandomForestRegressor` model, this approach improves the model's performance
     - the learning curves for this model suggest a low bias and high variance performance
       - the low training error backs up this claim of high variance
       - the validation curve does not plateau at the maximum tested training set size
         - it can still decrease and converge towards the training curve
       - the large gap between training and validation curves indicates overfitting and could be remedied using
         - added training and validation data, since both scores are trending towards eachother, and will converge with more data
         - reduced number of features, perhaps using feature selection
         - hyperparameter optimization
         - increased regularization, by increasing the `max_leaf_nodes` hyperparameter

<a id="conclusion-and-future-work"></a>

### 10. [Conclusion and Future work](#conclusion-and-future-work)

<a id="conclusion"></a>

#### 10.1. [Conclusion](#conclusion)

The goal of this project was to predict listed vehicle prices to within \$5,000. After data pre-processing and model development, the best model was trained on approx. 2,200 observations and made approx. 600 predictions.

The best model here scores an RMSE, on the predictions, of approx. \$4,900 (`R^2` = 0.61) which meets the goal for this project.

<a id="future-work"></a>

#### 10.1. [Future work](#future-work)

The limitation of this model is due to its moderate bias and low variance. The following are some areas where the current work could be expanded to explore improvement in model performance

1. As found from analysis of the learning curves for the best linear model, extract more features from the scraped data such as
   - additional attributes of the listing in the text of the description section such as (but not limited to)
     - horsepower
     - whether the car is a luxury brand or not
     - number of doors
     - financing options
   - explore non-linearities in newly extracted features
2. Numerical features were standardized by rescaling the distribution of their values so that the mean of observed values is 0 and the standard deviation is 1. A useful comparison could involve exploring the effect of normalized by rescaling the data so that all values are within the range of 0 and 1.
3. Considering that the five worst predictions of the model were in Washington state, more stringent removal of outliers should be explored on a per-feature basis and in the context of their relation to the listing prices. This is also warranted to explore the relationship between train_validation score and test set set score.i.e. to get a better estimate of the model's generalization error.
4. Feature selection techniques could be explored, using cross-validation, to decrease the number of input features.
5. Categorical features could be aggregated so that infrequently occurring features are gathered into a new category named `Other`. This could help reduce the cardinality of some of the categorical features. The disadvantage is that some of the predictive power of the feature could be lost since previously useful independent categories are combined into a possibly less useful aggregated one. The usefulness of this approach should be explored via cross-validation.